# 🌡️ Professional Sensor Analytics Dashboard

**Production-grade IoT monitoring with real-time data sync and advanced analytics**

Built with best practices for data visualization, accessibility, and user experience.

In [ ]:
!pip install firebase-admin gradio plotly scipy scikit-learn pandas numpy requests gdown -q

In [ ]:
# Download Firebase credentials
import gdown, os
firebase_key_file = 'firebase_key.json'
if not os.path.exists(firebase_key_file):
    print('📥 Downloading Firebase credentials...')
    try:
        gdown.download(f'https://drive.google.com/uc?id=15L_nwwjOXYZ1DwTZUc6Xk2oB7LH5lmSa', firebase_key_file, quiet=False)
        print('✓ Firebase credentials ready')
    except Exception as e:
        print(f'⚠️ Download failed: {e}. Upload manually.')
        from google.colab import files
        uploaded = files.upload()
        if uploaded: print('✓ File uploaded successfully!')
else:
    print('✓ Firebase key already available')

In [ ]:
# Import and initialize
import firebase_admin
from firebase_admin import credentials, db
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import gradio as gr
from datetime import datetime, timedelta
from scipy import stats
import requests, json, warnings
warnings.filterwarnings('ignore')

# Initialize Firebase
if not firebase_admin._apps:
    firebase_admin.initialize_app(credentials.Certificate('firebase_key.json'), {
        'databaseURL': 'https://cloud-81451-default-rtdb.europe-west1.firebasedatabase.app/'
    })
    print('✓ Firebase initialized')

# API Configuration
BASE_URL, FEED, BATCH_LIMIT = "https://server-cloud-v645.onrender.com/", "json", 200
print('✓ All packages loaded!')

In [ ]:
# Design system - Colorblind-safe Okabe-Ito palette
COLORS = {
    'temperature': {'primary': '#ef4444', 'gradient': 'linear-gradient(135deg, #dc2626 0%, #ef4444 100%)'},
    'humidity': {'primary': '#3b82f6', 'gradient': 'linear-gradient(135deg, #2563eb 0%, #3b82f6 100%)'},
    'soil': {'primary': '#10b981', 'gradient': 'linear-gradient(135deg, #059669 0%, #10b981 100%)'},
    'status': {'normal': '#10b981', 'warning': '#f59e0b', 'critical': '#ef4444', 'info': '#3b82f6'},
    'neutral': {'text': '#1f2937', 'subtext': '#6b7280', 'border': '#e5e7eb', 'bg': '#ffffff'}
}
# Sensor configuration for reusability
SENSORS = [
    ('temperature', '°C', COLORS['temperature']['primary'], COLORS['temperature']['gradient'], 'TEMPERATURE'),
    ('humidity', '%', COLORS['humidity']['primary'], COLORS['humidity']['gradient'], 'HUMIDITY'),
    ('soil', '%', COLORS['soil']['primary'], COLORS['soil']['gradient'], 'SOIL MOISTURE')
]
print('✓ Design system loaded')

In [ ]:
# Firebase sync functions
def get_latest_timestamp_from_firebase():
    try:
        latest = db.reference('/sensor_data').order_by_child('created_at').limit_to_last(1).get()
        return list(latest.values())[0]['created_at'] if latest else None
    except: return None

def fetch_batch_from_server(before_timestamp=None):
    params = {"feed": FEED, "limit": BATCH_LIMIT}
    if before_timestamp: params["before_created_at"] = before_timestamp
    try: return requests.get(f"{BASE_URL}/history", params=params, timeout=180).json()
    except: return {}

def save_to_firebase(data_list):
    if not data_list: return 0
    ref, saved = db.reference('/sensor_data'), 0
    for sample in data_list:
        try:
            vals = json.loads(sample['value'])
            # Validate and clip sensor values to valid ranges
            temperature = max(-50, min(100, float(vals['temperature'])))
            humidity = max(0, min(100, float(vals['humidity'])))
            soil = max(0, min(100, float(vals['soil'])))
            
            ref.child(sample['created_at'].replace(':', '-').replace('.', '-')).set({
                'created_at': sample['created_at'], 'temperature': temperature,
                'humidity': humidity, 'soil': soil
            })
            saved += 1
        except: continue
    return saved

def sync_new_data_from_server():
    msgs, latest = ["🔄 Starting sync..."], get_latest_timestamp_from_firebase()
    msgs.append(f"📊 Latest: {latest}" if latest else "📭 No existing data")
    resp = fetch_batch_from_server()
    if "data" not in resp:
        return "\n".join(msgs + ["❌ Error fetching data"]), 0
    new = [s for s in resp["data"] if not latest or s["created_at"] > latest]
    if new:
        saved = save_to_firebase(new)
        return "\n".join(msgs + [f"✨ Found {len(new)} new samples", f"✅ Saved {saved} records!"]), saved
    return "\n".join(msgs + ["✓ No new data"]), 0

print('✓ Sync functions loaded')

In [ ]:
# Data loading
def load_data_from_firebase():
    data = db.reference('/sensor_data').get()
    if not data: return pd.DataFrame()
    df = pd.DataFrame([{
        'timestamp': pd.to_datetime(v['created_at']),
        'temperature': float(v['temperature']),
        'humidity': float(v['humidity']),
        'soil': float(v['soil'])
    } for v in data.values()])
    df = df.sort_values('timestamp').reset_index(drop=True)
    
    # Data validation and cleaning
    # Clip humidity and soil moisture to valid range (0-100%)
    df['humidity'] = df['humidity'].clip(0, 100)
    df['soil'] = df['soil'].clip(0, 100)
    # Temperature reasonable range (-50 to 100°C for sensor data)
    df['temperature'] = df['temperature'].clip(-50, 100)
    
    return df

print('📥 Loading data...')
df = load_data_from_firebase()
print(f'✓ Loaded {len(df)} records')
if len(df) > 0:
    print(f'📅 Range: {df["timestamp"].min()} to {df["timestamp"].max()}')

In [ ]:
# Compact CSS
CUSTOM_CSS = f"""
@import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap');
* {{ font-family: 'Inter', sans-serif; }}
.kpi-card {{ background: white; padding: 24px; border-radius: 12px; box-shadow: 0 1px 3px rgba(0,0,0,0.12);
  text-align: center; transition: transform 0.2s; border-left: 4px solid; }}
.kpi-card:hover {{ transform: translateY(-4px); box-shadow: 0 3px 6px rgba(0,0,0,0.16); }}
.kpi-label {{ color: #6b7280; font-size: 14px; font-weight: 600; text-transform: uppercase; }}
.kpi-value {{ font-size: 48px; font-weight: 700; margin: 8px 0; color: #1f2937; }}
.kpi-change {{ font-size: 14px; font-weight: 600; }}
.trend-up {{ color: #10b981; }} .trend-down {{ color: #ef4444; }} .trend-stable {{ color: #3b82f6; }}
.stat-card {{ border-radius: 16px; padding: 24px; color: white; box-shadow: 0 3px 6px rgba(0,0,0,0.16); margin: 16px 0; position: relative; z-index: 1; overflow: visible !important; }}
.stat-card:has(.info-popup:hover) {{ z-index: 999999 !important; transform: translateZ(0); }}
.stat-item {{ background: rgba(255,255,255,0.2); border-radius: 12px; padding: 16px; position: relative; z-index: 1; }}
.info-popup {{ position: relative; display: inline-block; overflow: visible; }}
.info-text {{ visibility: hidden; width: 280px; background: rgba(0,0,0,0.95); color: white; text-align: left;
  border-radius: 8px; padding: 14px; position: absolute; z-index: 1000000; 
  top: 50%; transform: translateY(-50%); font-size: 13px; box-shadow: 0 6px 20px rgba(0,0,0,0.5);
  right: calc(100% + 10px); left: auto; }}
.info-text::before {{ content: ''; position: absolute; right: -8px; top: 50%; transform: translateY(-50%);
  border: 8px solid transparent; border-left-color: rgba(0,0,0,0.95); }}
.info-popup:hover .info-text {{ visibility: visible; }}
/* Adaptive positioning */
.stat-item:nth-child(odd) .info-text {{ left: calc(100% + 10px); right: auto; }}
.stat-item:nth-child(odd) .info-text::before {{ left: -8px; right: auto; 
  border-left-color: transparent; border-right-color: rgba(0,0,0,0.95); }}
@media (max-width: 768px) {{ .info-text {{ width: 220px; font-size: 12px; padding: 12px; }} }}
.status-badge {{ display: inline-flex; align-items: center; gap: 6px; padding: 6px 16px; border-radius: 20px;
  font-size: 14px; font-weight: 600; background: #dcfce7; color: #166534; }}
.status-dot {{ width: 8px; height: 8px; border-radius: 50%; background: #10b981; animation: pulse 2s infinite; }}
@keyframes pulse {{ 0%, 100% {{ opacity: 1; }} 50% {{ opacity: 0.5; }} }}
.explanation-card {{ background: {COLORS['temperature']['gradient']}; border-radius: 12px; padding: 16px;
  color: white; margin-bottom: 16px; box-shadow: 0 1px 3px rgba(0,0,0,0.12); }}
/* Ensure containers do not interfere */
/* Grid container must not create stacking context */
div[style*="display: grid"] {{ position: static; z-index: auto; }}
/* Stat items need z-index */
.stat-item {{ z-index: 1; }}
.stat-item:has(.info-popup:hover) {{ z-index: 1000000 !important; }}
"""
print('✓ CSS loaded')


In [ ]:
# Component functions
def create_kpi_card(label, value, unit, change, change_label, trend="up", border_color=None):
    bc = border_color or COLORS['status']['normal']
    icon = "↑" if trend == "up" else ("↓" if trend == "down" else "→")
    return f'''<div class="kpi-card" style="border-left-color: {bc};">
        <p class="kpi-label">{label}</p>
        <p class="kpi-value">{value}<span style="font-size: 24px;">{unit}</span></p>
        <p class="kpi-change trend-{trend}"><span>{icon}</span><span>{change} {change_label}</span></p>
    </div>'''

def create_status_badge(text="LIVE", pulse=True):
    return f'<span class="status-badge">{"<span class=\"status-dot\"></span>" if pulse else ""}{text}</span>'

def create_explanation_card(title, description, interpretation, gradient=None):
    return f'''<div class="explanation-card" style="background: {gradient or COLORS['temperature']['gradient']};">
        <h3>📊 {title}</h3><p><strong>What it shows:</strong> {description}</p>
        <p><strong>How to interpret:</strong> {interpretation}</p></div>'''

print('✓ Components loaded')

In [ ]:
# Statistics cards
def create_stat_cards_html(df):
    if len(df) == 0: return "<p>No data available</p>"
    
    explanations = {
        'Mean': 'Average value. Sum ÷ count.',
        'Median': 'Middle value. 50% above, 50% below.',
        'Std Dev': 'Variability around mean. Low=consistent, High=variable.',
        'Min': 'Lowest recorded value.',
        'Max': 'Highest recorded value.',
        'Q25': '25th percentile. 25% below this.',
        'Q75': '75th percentile. 75% below this.',
        'IQR': 'Interquartile range (Q75-Q25).'
    }
    
    
    html = '<div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 16px;">'
    for var, unit, _, grad, name in SENSORS:
        stats = {k: round(v, 2) for k, v in {
            'Mean': df[var].mean(), 'Median': df[var].median(), 'Std Dev': df[var].std(),
            'Min': df[var].min(), 'Max': df[var].max(),
            'Q25': df[var].quantile(0.25), 'Q75': df[var].quantile(0.75),
            'IQR': df[var].quantile(0.75) - df[var].quantile(0.25)
        }.items()}
        
        html += f'<div class="stat-card" style="background: {grad};"><h2>{name}</h2>'
        html += '<div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 14px;">'
        for stat_name, stat_val in stats.items():
            html += f'''<div class="stat-item"><div style="display: flex; justify-content: space-between;">
                <div style="font-size: 13px; font-weight: 500;">{stat_name}</div>
                <div class="info-popup" style="width: 20px; height: 20px; border-radius: 50%; border: 2px solid rgba(255,255,255,0.7);
                    display: flex; align-items: center; justify-content: center; font-size: 13px; cursor: help;
                    background: rgba(255,255,255,0.1);">i<span class="info-text">{explanations[stat_name]}</span></div>
                </div><div style="font-size: 26px; font-weight: 700;">{stat_val}{unit}</div></div>'''
        html += '</div></div>'
    html += '</div>'
    return html

print('✓ Statistics functions loaded')


In [ ]:
def distribution_analysis(df):
    """Histograms showing distribution of sensor values."""
    fig = make_subplots(rows=1, cols=3, 
                        subplot_titles=('Temperature (°C)', 'Humidity (%)', 'Soil Moisture (%)'))
    
    # Create histograms for each sensor
    for idx, (col, unit, color, _, name) in enumerate(SENSORS, 1):
        data = df[col].values
        data_min, data_max = data.min(), data.max()
        padding = (data_max - data_min) * 0.1
        bins = np.linspace(data_min - padding, data_max + padding, 31)
        counts, edges = np.histogram(data, bins=bins)
        centers = (edges[:-1] + edges[1:]) / 2
        
        fig.add_trace(go.Bar(
            x=centers, y=counts, name=name,
            marker_color=color,
            width=(data_max - data_min) / 30 * 0.9,
            hovertemplate=f'%{{x:.1f}}{unit}: %{{y}} readings<extra></extra>'
        ), row=1, col=idx)
        
        fig.update_xaxes(title_text=f"{name.capitalize()} ({unit})", row=1, col=idx)
        fig.update_yaxes(title_text="Number of Readings", row=1, col=idx)
    
    fig.update_layout(height=400, showlegend=False, plot_bgcolor='white', paper_bgcolor='white')
    
    return create_explanation_card(
        "Distribution Analysis",
        "Frequency of sensor values. Tall bars = common values, short bars = rare values.",
        "Look for the shape: bell curve = normal, multiple peaks = different patterns.",
        COLORS['humidity']['gradient']
    ), fig


In [ ]:
# Gradio Dashboard
def create_dashboard():
    with gr.Blocks(css=CUSTOM_CSS, theme=gr.themes.Soft(), title="Sensor Analytics") as demo:
        gr.Markdown("# 🌡️ Environmental Sensor Analytics Dashboard\n### Real-time monitoring with advanced statistical analysis")
        
        with gr.Row():
            gr.HTML(create_status_badge("LIVE", pulse=True))
            data_info = gr.Markdown(
                f"**📊 Data:** {len(df)} records" +
                (f" | **📅 Range:** {df['timestamp'].min()} to {df['timestamp'].max()}" if len(df) > 0 else "")
            )
        
        with gr.Row():
            sync_btn = gr.Button("🔄 Sync & Refresh", variant="primary", size="lg", scale=2)
            refresh_btn = gr.Button("↻ Refresh", variant="secondary", scale=1)
        sync_status = gr.Textbox(label="📡 Sync Status", lines=6, interactive=False)
        
        gr.Markdown("## 📈 Key Performance Indicators")
        with gr.Row(equal_height=True):
            if len(df) > 0:
                kpi_data = [(df[col].iloc[-1], df[col].mean(), col, unit, color) 
                           for col, unit, color, _, _ in SENSORS]
            else:
                kpi_data = [(0, 0, col, unit, color) for col, unit, color, _, _ in SENSORS]
            
            kpi_outputs = []
            for current, mean, col, unit, color in kpi_data:
                change = current - mean
                kpi_outputs.append(gr.HTML(create_kpi_card(
                    col.capitalize(), f"{current:.1f}", unit, f"{abs(change):.1f}{unit}",
                    "vs avg", "up" if change > 0 else "down", color
                )))
        
        with gr.Tab("📊 Statistics"): basic_cards = gr.HTML(create_stat_cards_html(df))
        with gr.Tab("📈 Time Series"): ts_explanation, ts_plot = gr.HTML(), gr.Plot()
        with gr.Tab("📉 Patterns"):
            hourly_explanation, hourly_plot = gr.HTML(), gr.Plot()
            gr.Markdown("---")
            daily_explanation, daily_plot = gr.HTML(), gr.Plot()
        with gr.Tab("🔗 Correlations"):
            corr_explanation, corr_plot = gr.HTML(), gr.Plot()
            gr.Markdown("---")
            scatter_explanation, scatter_plot = gr.HTML(), gr.Plot()
        with gr.Tab("📊 Distributions"): dist_explanation, dist_plot = gr.HTML(), gr.Plot()
        with gr.Tab("📊 Moving Avg"):
            ma_explanation = gr.HTML()
            var_selector = gr.Radio(['temperature', 'humidity', 'soil'], value='temperature', label="Variable")
            ma_plot = gr.Plot()
        with gr.Tab("⚠️ Anomalies"):
            gr.HTML(create_explanation_card("Anomaly Detection",
                "Z-score outlier detection (Z>3 = <0.3% probability).",
                "Investigate flagged measurements.", COLORS['status']['critical']))
            anomaly_table = gr.Dataframe(label="Anomalies", wrap=True)
        
        def sync_and_refresh():
            global df
            status, _ = sync_new_data_from_server()
            df = load_data_from_firebase()
            if len(df) > 0:
                info = f"**📊 Data:** {len(df)} records | **📅 Range:** {df['timestamp'].min()} to {df['timestamp'].max()}"
                kpi_updates = []
                for col, unit, color, _, _ in SENSORS:
                    current, mean = df[col].iloc[-1], df[col].mean()
                    change = current - mean
                    kpi_updates.append(create_kpi_card(col.capitalize(), f"{current:.1f}", unit,
                                                       f"{abs(change):.1f}{unit}", "vs avg",
                                                       "up" if change > 0 else "down", color))
                return (status, info, *kpi_updates, create_stat_cards_html(df),
                       *time_series_overview(df), *hourly_patterns(df), *daily_patterns(df),
                       *calculate_correlations(df), *scatter_analysis(df), *distribution_analysis(df),
                       *time_series_decomposition(df, 'temperature'), anomaly_detection(df))
            return (status, "**No data**") + ("",) * 16
        
        sync_btn.click(sync_and_refresh, outputs=[sync_status, data_info, *kpi_outputs, basic_cards,
                      ts_explanation, ts_plot, hourly_explanation, hourly_plot, daily_explanation, daily_plot,
                      corr_explanation, corr_plot, scatter_explanation, scatter_plot,
                      dist_explanation, dist_plot, ma_explanation, ma_plot, anomaly_table])
        var_selector.change(lambda v: time_series_decomposition(df, v) if len(df) > 0 else ("", go.Figure()),
                           inputs=[var_selector], outputs=[ma_explanation, ma_plot])
        
        if len(df) > 0:
            demo.load(lambda: (create_stat_cards_html(df), *time_series_overview(df), *hourly_patterns(df),
                              *daily_patterns(df), *calculate_correlations(df), *scatter_analysis(df),
                              *distribution_analysis(df), *time_series_decomposition(df, 'temperature'),
                              anomaly_detection(df)),
                     outputs=[basic_cards, ts_explanation, ts_plot, hourly_explanation, hourly_plot,
                             daily_explanation, daily_plot, corr_explanation, corr_plot,
                             scatter_explanation, scatter_plot, dist_explanation, dist_plot,
                             ma_explanation, ma_plot, anomaly_table])
    return demo

print('✓ Dashboard ready!')


In [ ]:
# Launch
print('🚀 Launching Dashboard...')
print('='*60)
print('Features: Real-time sync | KPI cards | Colorblind-safe | Material shadows')
print('          Tooltips | Axis labels | Stats annotations | Grid layout')
print('='*60)
demo = create_dashboard()
demo.launch(debug=True, share=True)